Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Zhiqi Chen"
COLLABORATORS = ""

---

# Homework 8: Expressions as Classes

Copyright Luca de Alfaro, 2019-20. 
License: [CC-BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/4.0/).

## Submission

[Please submit to this Google Form](https://docs.google.com/forms/d/e/1FAIpQLSeuHE5pmZV6soac2Psvcfn3TOyMZmNWN0ws3SQcs_tcguEOiw/viewform?usp=sf_link).

Deadline: Thursday November 12, 11pm (check on Canvas for updated information).

## Homework Format

There are four questions, each worth 10 points.  The questions ask you to write derivatives methods for the classes `V` (Variable), `Minus`, `Multiply` and `Divide`.

We will now describe a more sophisticated representation for an expression, based on a hierarchy of classes.  The class **Expr** is the generic class denoting an expression.  It is an _abstract_ class: only its subclasses will be instantiated.  For every operator, such as `+`, there will be a subclass, such as `Plus`.  Variables will correspond to a special subclass, called `V`.  Numerical constants will be just represented by numbers, and not by subclasses of `Expr`. 

## The `Expr` class

The `Expr` class implements the various [methods used to emulate numerical types](https://docs.python.org/3/reference/datamodel.html?#emulating-numeric-types), such as `__add__`, `__sub__`, and so forth.  In this way, we can build an expression simply by writing

    3 * V('x') / 2

Compared with the representation of expressions seen in the previous chapter, this class-based representation offers several advantages.  First, we can build expressions in a natural way as shown above, via the over-riding of the usual arithmetic operators.  Second, if we introduce a new operator, all we need is provide the implementation of the new operator: we do not need to modify the shared code that traverses the tree, and add one more case to a long case-analysis.  In other words, the code is far more modular.  This may seem a small point, but if one were to extend the representation of expressions to involve tensors (matrices) and operations on tensors, as is done in the symbolic representation of expressions used in machine-learning, the number of operators could easily grow to a hundred or more, making a modular approach the only reasonable one. 

### The `eval` method

We endow our class `Expr` with a method `eval`, which given a variable valuation -- a dictionary mapping variable names to values -- tries to compute the expression insofar as possible.  The method `eval` first evaluates the children $c_1, \ldots, c_n$ of the given expression, computing their values $v_1, \ldots, v_n$.  These values $v_i$, for $1 \leq i \leq n$, can be either _symbolic_, if they are an instance of `Expr`, or _numeric_.  They are symbolic if some of the variables in their subexpression was not assigned a value by our variable assignment.  For instance, if we evaluate

    (V('x') + 3) / V('y')

with respect to the assignment `{'x': 2}`, we obtain 

    5 / V('y')

which is still symbolic (an instance of `Expr`).  There are two cases: 

* If all $v_1, \ldots, v_n$ are numeric, `eval` calls the `op` method with arguments $v_1, \ldots, v_n$, to compute the numerical value of the expression.  The method `op` is not implemented in `Expr`, but rather, it is implemented in each operator `Plus`, `Minus`, etc, so as to compute the numerical value according to the operator. 

* If one or more of the results $v_1, \ldots, v_n$ are symbolic, then `eval` returns a new expression, with the same operator as the old one, but with children $v_1, \ldots, v_n$ rather than $c_1, \ldots, c_n$. 

We invite you to study the implementation of `eval` in detail, for the ideas used there will be used also in the implementation of other methods.

In [ ]:
class Expr(object):
    """Abstract class representing expressions"""

    def __init__(self, *args):
        self.children = list(args)
        self.child_values = None

    def eval(self, valuation=None):
        """Evaluates the value of the expression with respect to a given
        variable evaluation."""
        # First, we evaluate the children.
        child_values = [c.eval(valuation=valuation) if isinstance(c, Expr) else c
                        for c in self.children]
        # Then, we evaluate the expression itself.
        if any([isinstance(v, Expr) for v in child_values]):
            # Symbolic result.
            return self.__class__(*child_values)
        else:
            # Concrete result.
            return self.op(*child_values)

    def op(self, *args):
        """The op method computes the value of the expression, given the
        numerical value of its subexpressions.  It is not implemented in
        Expr, but rather, each subclass of Expr should provide its
        implementation."""
        raise NotImplementedError()

    def __repr__(self):
        """Represents the expression as the name of the class, followed by the
        children."""
        return "%s(%s)" % (self.__class__.__name__,
                        ', '.join(repr(c) for c in self.children))

    # Expression constructors

    def __add__(self, other):
        return Plus(self, other)

    def __radd__(self, other):
        return Plus(self, other)

    def __sub__(self, other):
        return Minus(self, other)

    def __rsub__(self, other):
        return Minus(other, self)

    def __mul__(self, other):
        return Multiply(self, other)

    def __rmul__(self, other):
        return Multiply(other, self)

    def __truediv__(self, other):
        return Divide(self, other)

    def __rtruediv__(self, other):
        return Divide(other, self)

Variables are expressions containing only one child, of type string, which is the name of the variable.  So `V("x")` will be a variable with name `"x"`.  The method `eval` of a variable over-rides that of `Expr`. 

In [ ]:
class V(Expr):
    """Variable."""

    def __init__(self, *args):
        """Variables must be of type string."""
        assert len(args) == 1
        assert isinstance(args[0], str)
        super().__init__(*args)

    def eval(self, valuation=None):
        """If the variable is in the evaluation, returns the
        value of the variable; otherwise, returns the expression."""
        if valuation is not None and self.children[0] in valuation:
            return valuation[self.children[0]]
        else:
            return self


Here are the constructors for the other operators; for them, we just need to provide an implementation for `op`.

In [ ]:
class Plus(Expr):
    def op(self, x, y):
        return x + y

class Minus(Expr):
    def op(self, x, y):
        return x - y

class Multiply(Expr):
    def op(self, x, y):
        return x * y

class Divide(Expr):
    def op(self, x, y):
        return x / y


We can build and evaluate expressions quite simply.

In [ ]:
e = V('x') + 3
print(e)
print(e.eval())
print(e.eval({'x': 2}))


Plus(V('x'), 3)
Plus(V('x'), 3)
5


In [ ]:
e = (V('x') + V('y')) * (2 + V('x'))
print(e.eval())
print(e.eval({'x': 4}))


Multiply(Plus(V('x'), V('y')), Plus(V('x'), 2))
Multiply(Plus(4, V('y')), 6)


Let's define `check_equal` as usual.

In [ ]:
def check_equal(x, y, msg=None):
    if x != y:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert x == y, "%r and %r are different" % (x, y)
    print("Success")

## Derivating an expression

We will develop here a method `derivate` such that, for an expression `e`, the method call `e.derivate('x')` returns the derivative of the expression with respect to `V('x')`.  To implement it, we note that to deriva a composite expression with respect to a variable, we need to derivate the subexpressions with respect to that same variable.  Thus, we divide the work as follows: 

* The method `derivate` will first call the method `derivate` for all the subexpressions, that is, the children of the expression, accumulating the results into a list `partials` (so called because these are partial derivatives). 
* Then, the method `derivate` will call `op_derivate`.  This method is an abstract method of `Expr`, and is implemented in each operator (`Plus`, `Minus`, etc).  The method `op_derivate` will compute the derivative of the expression, according to the operator type. 

Let us start by writing `derivate`.


In [ ]:
def expr_derivate(self, var):
    """Computes the derivative of the expression with respect to var."""
    partials = [(c.derivate(var) if isinstance(c, Expr) else 0)
                for c in self.children]
    return self.op_derivate(var, partials).eval()

Expr.derivate = expr_derivate

def expr_op_derivate(self, var, partials):
    raise NotImplementedError()

Expr.op_derivate = expr_op_derivate


In the above code, note that we have to take care to call the `derivate` method for a subexpression only if that subexpression has type `Expr`.  If the subexpression does not have type `Expr`, then it is a constant, and its derivative with respect to a variable will be 0. 

The last `eval()` in the above expression is simply an attempt to simplify the resulting expression.  

Let us now write `op_derivate` for `Plus`.  We have:

$$
\frac{\partial}{\partial x} (f + g) \:=\: \frac{\partial f}{\partial x} + \frac{\partial g}{\partial x}
$$

and translating this into code:

In [ ]:
def plus_op_derivate(self, var, partials):
    return Plus(partials[0], partials[1])

Plus.op_derivate = plus_op_derivate


Before we can test the code, we need to have some variables. What is the derivative of `V('x')` with respect to a variable?  If the variable is `x`, that is, the same variable appearing in the `V` constructor, then the derivative is 1, otherwise, it is 0.  This because for variables, 

$$
\frac{\partial x}{\partial x} = 1 \qquad \frac{\partial x}{\partial y} = 0
\; .
$$

We will let you translate this into code; it's a one-liner. 

## Question 1: derivative of a variable

In [ ]:
### Question 1: Definition of `V.derivate`

def variable_derivate(self, var):
    # YOUR CODE HERE
    if var==self.children[0]:
         return 1
    else:
        return 0

V.derivate = variable_derivate


In [ ]:
### Tests for Question 1: 10 points

e = V('x') + 3
print(e.derivate('x'))
check_equal(e.derivate('x'), 1)
check_equal(e.derivate('y'), 0)


1
Success
Success


We now let you define `op_derivate` for the other operators.  If you need to refresh your mind, [you can consult the Wikipedia page on derivatives](https://en.wikipedia.org/wiki/Derivative).

## Question 2: Derivative of subtraction

Define an `op_derivate` method for `Minus`, similarly to what we did for `Plus`. 

In [ ]:
### Question 2: write `Minus.op_derivate`

def minus_op_derivate(self, var, partials):
    # YOUR CODE HERE
    return Minus(partials[0], partials[1])

Minus.op_derivate = minus_op_derivate


In [ ]:
### Tests for `Minus.op_derivate`

e = V('x') - 4
check_equal(e.derivate('x'), 1)

e = 4 - V('x')
check_equal(e.derivate('x'), -1)



Success
Success


## Question 3: Define `op_derivate` for `Multiply`

Now you have to define the method `op_derivate` for a node of type `Multiply`. 
You may want to keep in mind the formula for the derivation of a product: 

$$
\frac{\partial}{\partial x} (f \cdot g) = 
\frac{\partial f}{\partial x} \cdot g + f \cdot \frac{\partial g}{\partial x} \; .
$$

In [ ]:
### Question 3: `Multiply.op_derivate`

def multiply_op_derivate(self, var, partials):
    """Please generate a formula exactly as described by the above math formula."""
    # YOUR CODE HERE
    return Plus(Multiply(variable_derivate(self.children[0], var), self.children[1]), Multiply(self.children[0], variable_derivate(self.children[1], var)))

Multiply.op_derivate = multiply_op_derivate


In [ ]:
### Question 3 Tests: 10 points, for `Multiply.op_derivate`

e = V('x') * V('y')
check_equal(e.derivate('x').eval(dict(x=3, y=2)), 2)

e = V('x') * V('x')
check_equal(e.derivate('x').eval(dict(x=5)), 10)



Success
Success


Why do we use the .eval() in the above assertion?  Because the problem is that, if you implement derivatives in the natural way, the expression you obtain is not simplified.  For instance, this is what we get:

In [ ]:
e.derivate('x')


Plus(Multiply(1, V('x')), Multiply(V('x'), 1))

## Question 4: Implement `op_derivate` for `Divide`.

Now you have to define the method `op_derivate` for a node of type `Divide`. 
You may want to keep in mind the formula for the derivation of a ratio: 

$$
\frac{\partial}{\partial x} \frac{f}{g} =
\frac{
\frac{\partial f}{\partial x} \cdot g - f \cdot \frac{\partial g}{\partial x}
}{g^2} \; .
$$

In [ ]:
### Question 4: implement `Divide.op_derivate`

def divide_op_derivate(self, var, partials):
    # YOUR CODE HERE
    return  Divide(Minus(Multiply(variable_derivate(self.children[0], var), self.children[1]), Multiply(self.children[0], variable_derivate(self.children[1], var))), Multiply(self.children[1],self.children[1]))


Divide.op_derivate = divide_op_derivate


In [ ]:
### Question 4, 10 points: Tests for `Divide.op_derivate`

e = V('x') / V('y')
check_equal(e.derivate('x').eval(dict(x=3, y=2)), 0.5)
check_equal(e.derivate('y').eval(dict(x=3, y=2)), -3 / 4)



Success
Success
